## Day 11: Dumbo Octopus

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day11/puzzle.ipynb)

[Problem statement](https://adventofcode.com/2021/day/11)

In [ ]:
#!value --name sampleRaw
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526

In [ ]:
#!value --name inputRaw
8826876714
3127787238
8182852861
4655371483
3864551365
1878253581
8317422437
1517254266
2621124761
3473331514

In [ ]:
type Octopus =
| Gaining of int
| Flashing
| Flashed

In [ ]:
let countFlashes grid = 
    grid
    |> Seq.cast<int*Octopus>
    |> Seq.map fst 
    |> Seq.sum

In [ ]:
#load "../formatting.fsx"
let private formatOctopus el =
    match el with
    | Gaining energy -> string energy
    | Flashing -> "🌟"
    | Flashed -> "<span style=\"filter: grayscale(80%);\">⭐</span>"

Formatter.Register<Octopus[,]>((fun (octopuses) -> formatTable formatOctopus octopuses), "text/html")
Formatter.Register<(int*Octopus)[,]>((fun (octopuses) -> $"flashes: {countFlashes octopuses}" + formatTable (snd >> formatOctopus) octopuses), "text/html")

In [ ]:
#!share sampleRaw --from value
#!share inputRaw --from value

#load "../common.fsx"

let parse input = 
    read2d input |> Array2D.map (fun c -> 0, Gaining (c |> string |> int))

let sampleGrid = parse sampleRaw |> displayPipe
let actualGrid = parse inputRaw

5,4,8,3,1,4,3,2,2,3
2,7,4,5,8,5,4,7,1,1
5,2,6,4,5,5,6,1,7,3
6,1,4,1,3,3,6,1,4,6
6,3,5,7,3,8,5,4,7,8
4,1,6,7,5,2,4,6,4,5
2,1,7,6,8,4,1,7,2,1
6,8,8,2,8,8,1,1,3,4
4,8,4,6,8,4,8,5,5,4
5,2,8,3,7,5,1,5,2,6


In [ ]:
let private inc (flashes, octo) =
    match octo with
    | Flashed -> flashes, Gaining 1
    | Gaining energy -> flashes, Gaining (1 + energy)
    | Flashing -> failwith "Flashing state was not expected"

let private triggerFlashing (flashes, octo) =
    match octo with
    | Gaining 10 -> flashes, Flashing
    | Gaining e -> flashes, Gaining e
    | _ -> flashes, octo

let private replyToFlashes (grid : (int*Octopus)[,]) i j (flashes, octo) =
    match octo with
    | Gaining energy ->
        let flashingAdjacent =
            [|
                // use slicing opertator in order not to check boundaries.
                grid.[i-1 .. i-1, j-1 .. j+1]
                grid.[i .. i, j-1 .. j-1]
                grid.[i .. i, j+1 .. j+1]
                grid.[i+1 .. i+1, j-1 .. j+1]
            |]
            |> Array.collect (Array2D.toArray)
            |> Array.map snd
            |> Array.filter ((=)Flashing)
        flashes, Gaining (min (energy + (flashingAdjacent |> Array.length)) 10)
    | _ -> flashes, octo

let private fade (flashes, octo) =
    match octo with
    | Flashing -> flashes, Flashed
    | _ -> flashes, octo

let private countFlashed (flashes, octo) =
    match octo with
    | Flashed -> flashes + 1, Flashed
    | Flashing -> flashes, failwith "Flashing state was not expected"
    | Gaining x -> flashes, Gaining x

let step (grid: (int*Octopus)[,]) =
    let incremented = Array2D.map (inc >> triggerFlashing) grid
    let rec flashingProcess g =
        let newGrid = 
            g |> Array2D.mapi (replyToFlashes g)
            |> Array2D.map fade
            |> Array2D.map triggerFlashing
        if (newGrid |> Array2D.toArray |> Array.forall (fun (_, octo) -> octo <> Flashing))
        then newGrid
        else flashingProcess newGrid
    flashingProcess incremented
    |> Array2D.map countFlashed

In [ ]:
let simulateInfinitely grid = 
    Seq.unfold (fun g ->
        let thisStepGrid = step g
        Some (thisStepGrid, thisStepGrid)
    ) grid
    |> Seq.append [grid]

In [ ]:
let private sample100 = 
    simulateInfinitely sampleGrid |> Seq.mapi (fun i x -> i,x)
    |> Seq.takeWhile (fun (i, _) -> i <= 100)
    |> Seq.toArray
    
Array.append sample100.[0..4] [|sample100.[^0]|]

index Item1 Item2 0 0 flashes: 0 5 4 8 3 1 4 3 2 2 3 2 7 4 5 8 5 4 7 1 1 5 2 6 4 5 5 6 1 7 3 6 1 4 1 3 3 6 1 4 6 6 3 5 7 3 8 5 4 7 8 4 1 6 7 5 2 4 6 4 5 2 1 7 6 8 4 1 7 2 1 6 8 8 2 8 8 1 1 3 4 4 8 4 6 8 4 8 5 5 4 5 2 8 3 7 5 1 5 2 6 1 1 flashes: 0 6 5 9 4 2 5 4 3 3 4 3 8 5 6 9 6 5 8 2 2 6 3 7 5 6 6 7 2 8 4 7 2 5 2 4 4 7 2 5 7 7 4 6 8 4 9 6 5 8 9 5 2 7 8 6 3 5 7 5 6 3 2 8 7 9 5 2 8 3 2 7 9 9 3 9 9 2 2 4 5 5 9 5 7 9 5 9 6 6 5 6 3 9 4 8 6 2 6 3 7 2 2 flashes: 35 8 8 ⭐ 7 4 7 6 5 5 5 5 ⭐ 8 9 ⭐ 8 7 ⭐ 5 4 8 5 9 7 8 8 9 6 ⭐ 8 8 4 8 5 7 6 9 6 ⭐ ⭐ 8 7 ⭐ ⭐ 9 ⭐ 8 8 ⭐ ⭐ 6 6 ⭐ ⭐ ⭐ 8 8 9 8 9 6 8 ⭐ ⭐ ⭐ ⭐ 5 9 4 3 ⭐ ⭐ ⭐ ⭐ ⭐ ⭐ 7 4 5 6 9 ⭐ ⭐ ⭐ ⭐ ⭐ ⭐ 8 7 6 8 7 ⭐ ⭐ ⭐ ⭐ 6 8 4 8 3 3 flashes: 80 ⭐ ⭐ 5 ⭐ 9 ⭐ ⭐ 8 6 6 8 5 ⭐ ⭐ 8 ⭐ ⭐ 5 7 5 9 9 ⭐ ⭐ ⭐ ⭐ ⭐ ⭐ 3 9 9 7 ⭐ ⭐ ⭐ ⭐ ⭐ ⭐ 4 1 9 9 3 5 ⭐ 8 ⭐ ⭐ 6 3 7 7 1 2 3 ⭐ ⭐ ⭐ ⭐ ⭐ 7 9 1 1 2 5 ⭐ ⭐ ⭐ 9 2 2 1 1 1 3 ⭐ ⭐ ⭐ ⭐ ⭐ 4 2 1 1 2 5 ⭐ ⭐ ⭐ ⭐ ⭐ 2 1 1 1 9 ⭐ ⭐ ⭐ 4 4 flashes: 96 2 2 6 3 ⭐ 3 1 9 7 7 ⭐ 9 2 3 ⭐ 3 1 6 9 7 ⭐ ⭐ 3 2 2 2 1 1 5 ⭐ ⭐ ⭐ 4 1 1 1 1 1 6 3 ⭐ ⭐ 7 6 1 9 1 1 7 4 ⭐ ⭐ 5 3 4 1 1 1 2 2 ⭐ ⭐ 4 2 3 6 1 1 2 ⭐ 5 5 3 2 2 4 1 1 2 2 1 5 3 2 2 4 7 2 1 1 1 1 3 2 2 3 ⭐ 2 1 1 5 100 flashes: 1656 ⭐ 3 9 7 6 6 6 8 6 6 ⭐ 7 4 9 7 6 6 9 1 8 ⭐ ⭐ 5 3 9 7 6 9 3 3 ⭐ ⭐ ⭐ 4 2 9 7 8 2 2 ⭐ ⭐ ⭐ 4 2 2 9 8 9 2 ⭐ ⭐ 5 3 2 2 2 8 7 7 ⭐ 5 3 2 2 2 2 9 6 6 9 3 2 2 2 2 8 9 6 6 7 9 2 2 2 8 6 8 6 6 6 7 8 9 9 9 8 7 6 6

In [ ]:
simulateInfinitely actualGrid
    |> Seq.skip 100
    |> Seq.head

1,1,1,3,2,2,7,5,4,8
1,1,5,3,2,2,2,6,4,4
1,6,3,2,2,2,3,8,5,4
1,5,2,2,2,3,5,1,8,6
1,5,2,2,3,5,1,1,1,1
1,4,2,3,5,1,1,1,1,1
3,4,4,6,1,1,1,1,1,1
9,9,1,2,3,3,2,1,1,7
8,1,2,4,⭐,⭐,4,2,7,5
7,1,2,⭐,⭐,⭐,⭐,7,5,4


In [ ]:
let findFirstAllFlashed grid = 
    simulateInfinitely grid 
    |> Seq.mapi (fun i x -> i, x)
    |> Seq.filter (fun (i, x) -> x |> Seq.cast<int*Octopus> |> Seq.forall (fun (_,o) -> o = Flashed))
    |> Seq.head

In [ ]:
let (private allFlashedStep, private allFlashed) = findFirstAllFlashed sampleGrid
allFlashedStep |> display
allFlashed |> display

195

⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐


In [ ]:
let (private allFlashedStep, private allFlashed) = findFirstAllFlashed actualGrid
allFlashedStep |> display
allFlashed |> display

788

⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐,⭐
